In [ ]:
import pandas as pd
import numpy as np
import camelot

## NIGC Gaming Tribe Report 2025

In [ ]:
tables_a = camelot.read_pdf('../data/TribesAlphabeticalList_NIGC_GamingTribeReport_2025.pdf',
                          flavor='stream',
                          row_tol=5,
                          pages='1-4')
report_a = tables_a[0].df
# report_a


In [138]:
tables_a_df_list = [table.df.iloc[3:, :] for table in tables_a]

In [197]:
report_df = pd.concat(tables_a_df_list, axis=0)
report_df

,0,1,2,3
3,Absentee Shawnee Tribe of Indians of OK,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Class Type: Class II & III See 25 CFR 502.3 & 4,Email Address: Governor@astribe.com
4,,Thunderbird Casino - Shawnee,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...
5,,Work Phone: 405-360-9270,Work Fax: 405-573-3211,Email Address: LisaF@TBECInc.com
6,,Thunderbird Entertainment Center,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...
7,,Work Phone: 405-360-9270,Work Fax: 405-360-9288,Email Address: LisaF@TBECInc.com
...,...,...,...,...
68,,Work Phone: 918-423-8161,Work Fax: 918-423-7938,Email Address: ltucker@choctawnation.com
69,,Choctaw Casino - Atoka Travel Plaza,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...
70,,Work Phone: 580-364-0056,Work Fax: 580-364-0057,Email Address: Tclem@choctawnation.com
71,,Choctaw Casino - Poteau Travel Plaza,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...


In [198]:
report_df.reset_index(drop=True, inplace=True)

In [199]:
is_nonempty = (report_df[0]!='').to_numpy()+0
# Find all indices where there are two 1's in a row
# This indicates a place where a tribe/nation name spilled over to two rows.
name_bleeds_over_idx = np.where((is_nonempty[:-1] == 1) & (is_nonempty[1:]==1))[0]
name_bleeds_over_idx

array([ 14,  32,  48,  51,  76,  88,  98, 103, 109, 131, 235, 245])

In [200]:
for i in name_bleeds_over_idx:
    report_df.loc[i, 0] += ' ' + report_df.loc[i+1, 0]
    report_df.loc[i+1, 0] = '' # No longer need the bleed-over text, it has been absorbed into the full name above


The `row_tol` parameter does not help us here since the multi-line tribe/nation name has the same distance between lines as the tribe/nation address and first facility name do. We may as well just keep all of these lines separate and rejoin the tribe/nation names based on adjacency and presence of a newline character. We then just extract the two-letter state abbreviation from the address with regular expressions. Are there any cases of a tribe/nation owning a facility in another state's borders?

Steps
1. Remove all rows that have empty column 0 AND have a column 1 that contains a row containing "Work Phone"
2. Stamp the tribe/nation name onto each of their facilities' names.
3. Extend to all pages of the pdf, not just the first page.

In [93]:
# report_a[1][report_a[1].str.contains('Work Phone')]

In [202]:
report_df = report_df.loc[~report_df[1].str.contains('Work Phone'), :]

In [203]:
report_df_clean = report_df.replace('', np.nan)

In [204]:
is_facility_idx = report_df_clean[0].isna()

In [205]:
report_df_clean['is_facility'] = is_facility_idx

In [206]:
report_df_clean[0] = report_df_clean[0].ffill()

In [207]:
report_df_clean

,0,1,2,3,is_facility
0,Absentee Shawnee Tribe of Indians of OK,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Class Type: Class II & III See 25 CFR 502.3 & 4,Email Address: Governor@astribe.com,False
1,Absentee Shawnee Tribe of Indians of OK,Thunderbird Casino - Shawnee,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...,True
3,Absentee Shawnee Tribe of Indians of OK,Thunderbird Entertainment Center,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...,True
5,Agua Caliente Band of Cahuilla Indians,"5401 Dinah Shore Dr., Palm Springs CA 92264",Class Type: Class II & III See 25 CFR 502.3 & 4,Email Address: laviles@aguacaliente.net,False
6,Agua Caliente Band of Cahuilla Indians,Agua Caliente Casino Rancho Mirage,"5401 Dinah Shore Dr., Palm Springs CA 92264",Game Class Type: Class II & III See 25 CFR 5...,True
...,...,...,...,...,...
271,Choctaw Nation of Oklahoma,Choctaw Casino - Grant Travel Plaza,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...,True
273,Choctaw Nation of Oklahoma,Choctaw Casino - Stigler,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...,True
275,Choctaw Nation of Oklahoma,Choctaw Casino - McAlester,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...,True
277,Choctaw Nation of Oklahoma,Choctaw Casino - Atoka Travel Plaza,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...,True


In [208]:
# report_df_clean.to_csv('../data/raw_gaming_report.csv')

In [210]:
# Addresses are in different columns for facilities and non-facilities
report_df_clean['address'] = np.where(report_df_clean['is_facility'], report_df_clean[2], report_df_clean[1])

In [212]:
report_df_clean['classes_raw'] = np.where(report_df_clean['is_facility'], report_df_clean[3], report_df_clean[2])

In [214]:
report_df_clean['facility_name'] = np.where(report_df_clean['is_facility'], report_df_clean[1], np.nan)

In [217]:
report_df_clean.drop(columns=[1,2,3], inplace=True)

In [222]:
report_df_clean['state'] = report_df_clean['address'].str.extract(r'([A-Z]{2})')

In [228]:
report_df_clean['classes_raw'].str.contains('Class III')

0      False
1      False
3      False
5      False
6      False
       ...  
271    False
273    False
275    False
277    False
279    False
Name: classes_raw, Length: 161, dtype: object

In [229]:
report_df_clean['classes'] = ''

In [233]:
report_df_clean['classes_raw'].str.contains('Class II & III')

0      True
1      True
3      True
5      True
6      True
       ... 
271    True
273    True
275    True
277    True
279    True
Name: classes_raw, Length: 161, dtype: object

In [235]:
report_df_clean['classes_raw'].unique()

array(['Class Type: Class II & III   See 25 CFR 502.3 & 4',
       'Game Class Type:  Class II & III  See 25 CFR 502.3 & 4',
       'Game Class Type:  Class II  See 25 CFR 502.3 & 4',
       'Game Class Type:  Class III  See 25 CFR 502.3 & 4',
       'Class Type: Class II   See 25 CFR 502.3 & 4',
       'P.O. Box 337 37302 Rancheria Lane, Auberry CA 93602 Game Class Type:  Class III  See 25 CFR 502.3 & 4',
       'Class Type: Class III   See 25 CFR 502.3 & 4', nan,
       'tribalcouncil@cahuilla-nsn.gov',
       'Class Type:   See 25 CFR 502.3 & 4'], dtype=object)

In [ ]:
# report_df_clean.to_csv('../data/raw_gaming_report.csv')
# Save, then manually fix the extra row spaces for:
    # Big Sandy Band of Western Mono Indians
    # Bois Forte
    # Cahuilla Band
    # Campo Band of Diegueno Mission of the Campo Indian Reservation
    # Chicken Ranch Rancheria of Me-wuk Indians of California
    # More?


In [243]:
report_df_clean = pd.read_csv('../data/raw_gaming_report.csv').iloc[:,1:]

In [244]:
report_df_clean

,0,is_facility,address,classes_raw,facility_name,state,classes
0,Absentee Shawnee Tribe of Indians of OK,False,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Class Type: Class II & III See 25 CFR 502.3 & 4,NaN,OK,NaN
1,Absentee Shawnee Tribe of Indians of OK,True,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...,Thunderbird Casino - Shawnee,OK,NaN
2,Absentee Shawnee Tribe of Indians of OK,True,"2025 S. Gordon Cooper Dr., Shawnee OK 74801",Game Class Type: Class II & III See 25 CFR 5...,Thunderbird Entertainment Center,OK,NaN
3,Agua Caliente Band of Cahuilla Indians,False,"5401 Dinah Shore Dr., Palm Springs CA 92264",Class Type: Class II & III See 25 CFR 502.3 & 4,NaN,CA,NaN
4,Agua Caliente Band of Cahuilla Indians,True,"5401 Dinah Shore Dr., Palm Springs CA 92264",Game Class Type: Class II & III See 25 CFR 5...,Agua Caliente Casino Rancho Mirage,CA,NaN
...,...,...,...,...,...,...,...
149,Choctaw Nation of Oklahoma,True,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...,Choctaw Casino - Grant Travel Plaza,OK,NaN
150,Choctaw Nation of Oklahoma,True,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...,Choctaw Casino - Stigler,OK,NaN
151,Choctaw Nation of Oklahoma,True,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...,Choctaw Casino - McAlester,OK,NaN
152,Choctaw Nation of Oklahoma,True,"P.O. Drawer 1210, Durant OK 74702",Game Class Type: Class II & III See 25 CFR 5...,Choctaw Casino - Atoka Travel Plaza,OK,NaN


In [ ]:
# Since so many state values were on subsequent empty lines, must update after manually cleaning those problem rows
report_df_clean['state'] = report_df_clean['address'].str.extract(r'([A-Z]{2})')

In [ ]:
report_df_clean.loc[:, 'classes'] = 'Other'
report_df_clean.loc[report_df_clean['classes_raw'].str.contains('Class II'), 'classes'] = 'Class II'
report_df_clean.loc[report_df_clean['classes_raw'].str.contains('Class III'), 'classes'] = 'Class III'
report_df_clean.loc[report_df_clean['classes_raw'].str.contains('Class II & III'), 'classes'] = 'Class II & III'



The Catawba Indian Nation operates [***one*** Class III facility](https://www.americangaming.org/wp-content/uploads/2025/02/North-Carolina_AGA-Gaming-Regulatory-Fact-Sheet-2025.pdf) in North Carolina. There is a typo in the pdf saying that the facility is in South Carolina, but that is just where the nation is based.

In [275]:
report_df_clean.loc[(report_df_clean.iloc[:,0]=='Catawba Indian Nation') & (~report_df_clean['is_facility']),
                     'classes'] = 'Class III'

In [279]:
report_df_clean.loc[(report_df_clean.iloc[:,0]=='Catawba Indian Nation') & (report_df_clean['is_facility']),
                     'address'] = '538 Kings Mountain Blvd, Kings Mountain, NC 28086'


In [280]:
report_df_clean.loc[(report_df_clean.iloc[:,0]=='Catawba Indian Nation') & (report_df_clean['is_facility']),
                     'state'] = 'NC'


In [278]:
report_df_clean['classes'].value_counts()

classes
Class II & III    113
Class III          24
Class II           17
Name: count, dtype: int64

In [283]:
report_df_clean.drop(columns=['classes_raw'], inplace=True)

In [287]:
report_df_clean.rename(columns={'0':'owner_name'}, inplace=True)

In [ ]:
# report_df_clean.loc[
#     :, ['facility_name', 'owner_name', 'is_facility', 'address', 'state', 'classes']
#     ].to_csv('../data/clean_gaming_report.csv')

### Load from here

In [11]:
report_df_clean = pd.read_csv('../data/clean_gaming_report.csv')

In [12]:
owner_names = report_df_clean['owner_name'].unique().tolist()

I really only care about a handful of items from the NIGC gaming report PDF:
* Address of the gaming facility
    * Extract the state as well
* Name of tribe or nation that owns the facility
* Name of the facility
* Class type (Could be a string with some junk in it, will extract Class number later)
* For now, don't really need all the contact info

## Native Economic Trends

In [14]:
import pandas as pd
import difflib

In [5]:
econ = pd.read_excel('../data/native-economic-trends-data_current.xlsx', sheet_name=1)

In [13]:
geonames = econ['geoname_mostrecent'].unique().tolist()

I was considering just using ChatGPT or something to automatically create a crosswalk between the `owner_name` column from the NIGC table and `geoname_mostrecent` here. I should be responsible about validating this crosswalk.

`difflib` should do the trick, too.

In [25]:
best_geonames = []
for owner_name in owner_names:
    best = difflib.get_close_matches(owner_name, geonames, n=1, cutoff=0.42)
    best_geonames.append(best[0] if best else None)

In [27]:
any([item is None for item in best_geonames]) # Every owner name had a match!

False

In [29]:
owner_land_crosswalk = pd.DataFrame({'owner_name': owner_names, 'geoname_mostrecent':best_geonames})

In [31]:
owner_land_crosswalk.shape

(34, 2)